In [ ]:
import yfinance as yf
import pandas as pd
import os

def get_financial_data(ticker):
    """Get and combine financial statements for a single ticker"""
    try:
        df = yf.Ticker(ticker)
        
        # Get financial statements
        income_statement = df.income_stmt
        balance_sheet = df.balance_sheet
        cash_flow = df.cashflow
        
        # Combine financial statements
        financials_statement = pd.concat([income_statement, balance_sheet, cash_flow], axis=0)
        
        # Reorder columns - format the columns to YYYY format and rearrange for the newest year first
        if not financials_statement.empty:
            # Convert column names to proper datetime and sort
            financials_statement.columns = pd.to_datetime(financials_statement.columns).strftime('%Y')
            financials_statement = financials_statement.sort_index(axis=1, ascending=False)
            # Format columns to YYYY only if desired
            financials_statement.columns = financials_statement.columns.str.split('-').str[0]
        
        return financials_statement
        
    except Exception as e:
        print(f"❌ Error retrieving data for {ticker}: {e}")
        return None

def process_single_ticker(ticker):
    """Process a single ticker"""
    financials_statement = get_financial_data(ticker)
    
    if financials_statement is not None:
        print(f"\n📊 Financial Data for {ticker}:")
        display(financials_statement)
        
        # Save to CSV
        filename = f"financial_data/{ticker}_financials.csv"
        financials_statement.to_csv(filename, encoding='utf-8')
    else:
        print(f"Failed to process {ticker}")

def process_multiple_tickers(csv_file):
    """Process multiple tickers from a CSV file"""
    try:
        # Read CSV file
        tickers_df = pd.read_csv(csv_file)
        
        # Assume tickers are in a column named 'ticker' or first column
        if 'ticker' in tickers_df.columns:
            tickers = tickers_df['ticker'].tolist()
        else:
            tickers = tickers_df.iloc[:, 0].tolist()
        
        for ticker in tickers:
            ticker = str(ticker).strip()
            if ticker:  # Skip empty tickers
                process_single_ticker(ticker)
                
    except Exception as e:
        print(f"❌ Error processing CSV file: {e}")

def main():
    while True:
        user_input = input("\nEnter ticker symbol, CSV file path, or 'q' to quit: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            break
        
        if not user_input:
            continue
        
        # Check if input is a CSV file
        if user_input.lower().endswith('.csv'):
            if os.path.exists(user_input):
                process_multiple_tickers(user_input)
            else:
                print(f"❌ CSV file not found: {user_input}")
        else:
            # Process single ticker
            process_single_ticker(user_input)

# Run the program
if __name__ == "__main__":
    main()